### 1. 라이브러리 로드

In [122]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import os
os.chdir('D:\kidney')

### 2. 크롬 드라이브로 사이트 열었다 닫기


###  3 테스트 추출

##### 3-1 드라이버 실행

In [12]:
# 크롬드라이버 실행
driver = webdriver.Chrome('D:\kidney\driver/chromedriver')
# 3초 대기
driver.implicitly_wait(3)# 웹브라우져 3초 대기 #묵시적인 웨이팅 시간을 주는것 관례상 3초간 대기시간을 줌 넣으주는것이 좋음.
# 웹 브라우져 오픈
driver.get('http://www.snuhlab.org/checkup/check_list.aspx?page={}&'.format(1))

##### 3-2 함수정의
##### 1. 리스트 에서 중복되는 컬럼명을 제거하고, 
##### 2. 문자열 -> 문자들로 구분 -> 문자열로 변경

In [95]:
def val_split_join(input_lst):
    split_col_value_lst = input_lst.split('\n')
   
    val_lst = []
    for num2 in range(0,19):
        values= split_col_value_lst[num2].split(' ')[1:]
        values_join = ' '.join(values)
        val_lst.append(values_join)
    return val_lst

In [123]:
def page_crawling():
    # dataframe 만들기
    df_col = ['검사분류코드', '검사분류명', '검사항목코드', '검사항목명', '참고치', '단위', '기본검체', '검체용기 이미지',
       '검체용기', '주의사항', '시행일', '접수마감시간', '보고소요시간', '검사방법', '검사의의', '비고', '동의어',
       '오더가능여부', '시행여부']
    df = pd.DataFrame(columns=df_col)
    df
    
    
    # 검사명 클릭을 위한 설정
    strong_list = driver.find_elements_by_css_selector('strong')

    #for i in range(len(strong_list)):
    for i in range(10):
        strong_list[i].click() #검사명 클릭
        time.sleep(4)
        td_lst = driver.find_elements_by_css_selector('thead')

        #
        raw_lst =[]
        for num in range(10):
            data = td_lst[num].text
            raw_lst.append(data)

        # ''총 10개중 9개가 순서대로 들어있다. 제거한다. 
        if i < 9:
            col_value_lst = raw_lst[i:-(9-i)][0]
        else:
            col_value_lst = raw_lst[i:][0]

        split_col_value_lst = col_value_lst.split('\n')
    #    print(split_col_value_lst, sep='\n')
    #    print(i, len(split_col_value_lst))

        #val_split_join()
    #    val_split_join(col_value_lst)
        val_lst = []
        for num2 in range(0,19):
            values= split_col_value_lst[num2].split(' ')[1:]
            values_join = ' '.join(values)
            val_lst.append(values_join)

        print(val_lst)
        df = df.append(pd.Series(val_lst, index=df.columns), ignore_index=True)
        time.sleep(4)
    return df

##### 3-3 함수 실행

##### 함수 실행하고 리턴 받는 방법
df = page_crawling()

In [124]:
# 구글 웹브라우져 새창이 뜬다.
# dataframe 만들기
df_col = ['검사분류코드', '검사분류명', '검사항목코드', '검사항목명', '참고치', '단위', '기본검체', '검체용기 이미지',
   '검체용기', '주의사항', '시행일', '접수마감시간', '보고소요시간', '검사방법', '검사의의', '비고', '동의어',
   '오더가능여부', '시행여부']
df_all = pd.DataFrame(columns=df_col)


# 전체 페이지 크롤링
driver = webdriver.Chrome('D:\kidney\driver/chromedriver')
for num in range(1,156):
#    driver = webdriver.Chrome('D:\kidney\driver/chromedriver')
    driver.implicitly_wait(3)# 웹브라우져 3초 대기 #묵시적인 웨이팅 시간을 주는것 관례상 3초간 대기시간을 줌 넣으주는것이 좋음.
    driver.get('http://www.snuhlab.org/checkup/check_list.aspx?page={}&'.format(num))
    time.sleep(5)
    
    # 한 페이지 크롤링 함수 삽입
    df = page_crawling()
    
    # 데이터 프레임 결합
    df_all = pd.concat([df_all, df]) 
    
    
df_all

['L43', '미생물특수', 'L4326', 'Adenovirus IFA', 'Negative', '', 'Endotracheal Aspirate', '이미지', 'ETA용기', '면봉검체는 환부의 세포를 많이 취하여 IFA용 슬라이드의 구멍에 부드럽게 펴 바른다. [IFA용 슬라이드는 검사실에서 불출]', '월-금', '오후 1시', '1~2일', '직접 면역 형광 염색', '폐렴 등 호흡기 감염을 주로 일으키는 혈청형의 adenovirus 항원을 검출. 대변 검체의 경우는 Adenovirus antigen 검사로 의뢰할 것(검출되는 혈청형이 다름)', '', '형광염색, IFA, DFA', '예', '시행중']
['L43', '미생물특수', 'L4322', 'Adenovirus antigen', 'Negative', '', 'Stool', '이미지', '대변통', '', '월-금 / 토-일', '오후 5시 / 오후 3시', '1~2일 / 2~3일', 'ELISA', '바이러스성 설사', '월-금 [오전9시] 접수된 검체는 당일 보고', 'Adenovirus, 아데노, 바이러스, 설사', '예', '시행중']
['L22', '혈액응고', 'L2204', 'aPTT(activated partial thromboplastin time)', '27.1 ~37.8 sec', 'sec', 'Citrate blood 2.7mL', '이미지', 'vacutainer(Sodium citrate, 현재 2.7 mL 공급)', '환자의 Hct 이 55% 이상인 경우: 응고검사실에 연락하여 항응고제농도가 조절된 채혈관을 받아가서 채혈해야 함', '매일', '24시간 검사', '2시간(본원) / 1시간(암병원)', 'Turbidimetry', 'activated partial thromboplastin time (aPTT)은 intrinsic pathway 를 평가하는 screening test이다. aPTT는 factor VII, XIII을 제외한 모든 인자들이 관여하는 검사

['L71', '외주검사', 'L7107', '17-KS(외주)', '남 년도별 연령 : 0~7 참고치 : 0.0~3.0 여 년도별 연령 : 0~7 참고치 : 0.0~3.0 남 년도별 연령 : 0~8 참고치 : 0~3.0 여 년도별 연령 : 0~8 참고치 : 0~3.0 남 년도별 연령 : 9~999 참고치 : 10~25 여 년도별 연령 : 9~999 참고치 : 7~20', 'mg/day', '24h UR 20 ml', '이미지', '30. 24H Urine 20mL', '냉장 무방부제', '화,금', '검사 시일행 전날 오후 3시', '3일', 'Colorimetry', '부신피질암, 부신과형성, 쿠싱병, 스트레스, 고환암에서 증가하며, Addison 병, 뇌하수체 기능저하증, 갑상 선 기능저하증에서 감소한다. (Cushing 증후군, 부신성기증후군, 기타 다른 steroid biosynthetic defect 가 의심되는 환자의 진단에 이용)', '1.위탁처:녹십자의뢰 외주검사 (☏.1566-0131) 2.주관부서:특수화학 검사실 3.수가코드:L7107 4.오더(그룹)코드 5.지역채혈:해당없음 6.도움말 수정이력', '17-ketosteroid', '예', '시행중']
['L71', '외주검사', 'L7327', '17-OHCS(외주)', '남 년도별 연령 : 0~999 참고치 : 5~23 여 년도별 연령 : 0~999 참고치 : 3~15', 'mg/day', '24h UR 20 ml', '이미지', '30. 24H Urine 20mL', '냉장, 무방부제', '화, 금', '검사 시행일 전날 오후 3시', '3일', '비색법', 'cushing syndrom, 이소성 ACTH증후군, 스트레스성 당뇨, 갑상선 기능항진증, 말단비대증, 임신 등에서 증가하며, Addison 병, 급성부신부전증, 속발성 부신부전증 등에서 감소한다.', '녹십자의뢰 외주검사(☏.578-0131)', '17-Hydroxy corticosteroid', '예', '시행중']
['L72

['L25', '분자유전학', 'L25435', '가족검사 Arylsulfatase A Deficiency, ARSA sequencing', 'Negative', '', 'EDTA blood 6 mL', '이미지', '', '1일내 실온 보관 및 운송, 유전자검사 동의서', '월-토', '월-금(오후5시),토(오후12시30분)', '4주(추가검사 필요시 6주)', 'PCR, Sequencing', '1. 유전자: ARSA [arylsulfatase A; 22q13.33; GeneID 410] 2. 분자진단검사: 환자에서 시행한 ARSA sequencing 검사에서 돌연변이가 발견된 경우, 환자의 가족에서 그 돌연변이 유무를 검사함. 3. 관련 질환: Arylsulfatase A Deficiency (Metachromatic Leukodystrophy)', '[2012.12.17] 신규검사등록', 'Metachromatic Leukodystrophy', '예', '시행중']


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.123)


In [118]:
df_all.to_csv('data/test_서울대학교진단의학과.csv', encoding= 'utf-8-sig', index= False)
df_all

,검사분류코드,검사분류명,검사항목코드,검사항목명,참고치,단위,기본검체,검체용기 이미지,검체용기,주의사항,시행일,접수마감시간,보고소요시간,검사방법,검사의의,비고,동의어,오더가능여부,시행여부
0,L43,미생물특수,L4326,Adenovirus IFA,Negative,,Endotracheal Aspirate,이미지,ETA용기,면봉검체는 환부의 세포를 많이 취하여 IFA용 슬라이드의 구멍에 부드럽게 펴 바른다...,월-금,오후 1시,1~2일,직접 면역 형광 염색,폐렴 등 호흡기 감염을 주로 일으키는 혈청형의 adenovirus 항원을 검출. 대...,,"형광염색, IFA, DFA",예,시행중
1,L43,미생물특수,L4322,Adenovirus antigen,Negative,,Stool,이미지,대변통,,월-금 / 토-일,오후 5시 / 오후 3시,1~2일 / 2~3일,ELISA,바이러스성 설사,월-금 [오전9시] 접수된 검체는 당일 보고,"Adenovirus, 아데노, 바이러스, 설사",예,시행중
2,L22,혈액응고,L2204,aPTT(activated partial thromboplastin time),27.1 ~37.8 sec,sec,Citrate blood 2.7mL,이미지,"vacutainer(Sodium citrate, 현재 2.7 mL 공급)",환자의 Hct 이 55% 이상인 경우: 응고검사실에 연락하여 항응고제농도가 조절된 ...,매일,24시간 검사,2시간(본원) / 1시간(암병원),Turbidimetry,activated partial thromboplastin time (aPTT)은 ...,Unfractionated Heparin 치료모니터링을 위해 사용하는 aPTT의 치...,,예,시행중
3,L22,혈액응고,L2265,D-dimer assay (정량),0.04~0.49ug/mL (FEU),ug/mL (FEU),Citrate blood 2.7mL,이미지,"vacutainer(Sodium citrate, 현재 2.7 mL 공급)",,매일,24시간(본원) / 월-금 17시(암병원),4시간(본원) / 1시간(암병원),Immunoturbidimetry,D-dimer는 fibrin degradation product에 매우 특이적인 검...,,,예,시행중
4,L43,미생물특수,L4338,Influenza A/B virus 신속항원검사,Negative,,"Nasopharyngeal Swab, Endotracheal aspirate",이미지,"Virus 수송배지, ETA 용기",,월-금 / 토-일,오후4시 / 오후3시,당일 / 당일,Immuno-chromatography Assay,Influenza virus 감염을 선별,,"Influenza virus, 신속항원검사, 인플루엔자 항원",예,시행중
5,L43,미생물특수,L4341,Legionella urinary antigen test,Negative,,urine,이미지,"urine cup, conicaltube 15mL",,월-금/ 토-일,오후 4시 / 오후 3시,당일 / 1~2일,Immunochromatography assay,소변에서 L. pneumophila serogroup 1 의 항원을 검출하여 폐렴을...,,"Legionella,레지오넬라 항원",예,시행중
6,L71,외주검사,L74903,(1-3)-β-D-Glucan Quantitative Test,남 년도별 연령 : 0~999 참고치 : ~<60.0 여 년도별 연령 : 0~999...,pg/mL,Serum only,이미지,14. Serum-4.0,분리 후 SST 그대로 냉동 (cap open 금지),월~금,검사 시행일 전날 오후 3시,3일,Colorimetry,"(1-3)-β-D-Glucan 정량검사는 주요 진균 감염인 Candida spp.,...",[참고치변경 2019.1.14~] Negative : <60.0 Intermedia...,,예,시행중
7,L72,분당의뢰검사,L38020,(검진용)NST(Hunter),,,Blood Spot,이미지,13. Blood spot,,목,검사 시행일 전날 오전 11시,21일,LC-MS/MS,선천성 대사이상 질환을 신생아기에 스크리닝하여 조기 치료 및 예장할 수 있는 검사,2018.05.08 처방오픈,,예,시행중
8,L72,분당의뢰검사,L38022,"(검진용)NST(LSD, Lysosomal Storage Disease)",,,Blood spot,이미지,13. Blood spot,,목,검사 시행일 전날 오전 11시,21일,LC-MA/MS,선천성 대사이상 질환을 신생아기에 스크리닝하여 조기 치료 및 예장할 수 있는 검사,2018.05.08 처방오픈,,예,시행중
9,L71,외주검사,L7101,"1,25(OH)2 vitamin D3(외주)",남 년도별 연령 : 0~999 참고치 : 19.6~54.3 여 년도별 연령 : 0~...,pg/ml,Serum 4 ml,이미지,14. Serum-4.0,냉장운송 원심분리후 냉동,수,검사 시행일 전날 오후 3시,5일,RIA,Vitamin D3 는 전구물질 7-dehydrocholesterol 의 자외선 조...,녹십자의뢰 외주검사 (☏.578-0131),,예,시행중
